In [3]:
import requests
import json
import csv
import os

# Define the API links and corresponding file names
api_links = {
    "view": "https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if",
    "like": "https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if",
    "inspire": "https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if",
    "rating": "https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if",
    "summary": "https://api.socialverseapp.com/posts/summary/get?page=1&page_size=1000",
    "users": "https://api.socialverseapp.com/users/get_all?page=1&page_size=1000"
}

# Authentication headers (if required)
headers = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

# Create directories for JSON and CSV files
os.makedirs("json_files", exist_ok=True)
os.makedirs("csv_files", exist_ok=True)

# Function to save JSON to CSV
def json_to_csv(json_data, csv_file_path):
    posts = json_data.get("posts", json_data.get("users", []))  # Use 'posts' or 'users' key
    if not posts:
        print(f"No data found to save for {csv_file_path}")
        return

    with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        # Write the header from keys of the first item
        writer.writerow(posts[0].keys())
        # Write rows of data
        for item in posts:
            writer.writerow(item.values())

# Process each API link
for name, url in api_links.items():
    try:
        print(f"Fetching data from {name} API...")
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            json_data = response.json()
            json_file_path = f"json_files/{name}.json"
            csv_file_path = f"csv_files/{name}.csv"

            # Save JSON data to file
            with open(json_file_path, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, ensure_ascii=False, indent=4)
            print(f"Saved JSON data to {json_file_path}")

            # Convert JSON to CSV
            json_to_csv(json_data, csv_file_path)
            print(f"Saved CSV data to {csv_file_path}")
        else:
            print(f"Failed to fetch data from {name} API. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while processing {name} API: {e}")

print("Processing completed for all APIs.")


Fetching data from view API...
Saved JSON data to json_files/view.json
Saved CSV data to csv_files/view.csv
Fetching data from like API...
Saved JSON data to json_files/like.json
Saved CSV data to csv_files/like.csv
Fetching data from inspire API...
Saved JSON data to json_files/inspire.json
Saved CSV data to csv_files/inspire.csv
Fetching data from rating API...
Saved JSON data to json_files/rating.json
Saved CSV data to csv_files/rating.csv
Fetching data from summary API...
Saved JSON data to json_files/summary.json
Saved CSV data to csv_files/summary.csv
Fetching data from users API...
Saved JSON data to json_files/users.json
Saved CSV data to csv_files/users.csv
Processing completed for all APIs.


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

class EnhancedVideoRecommender:
    def __init__(self):
        # Load data
        self.likes = pd.read_csv('csv_files/like.csv')
        self.ratings = pd.read_csv('csv_files/rating.csv')
        self.users = pd.read_csv('csv_files/users.csv')
        self.views = pd.read_csv('csv_files/view.csv')
        self.summary = pd.read_csv('csv_files/summary.csv')
        self.inspire = pd.read_csv('csv_files/inspire.csv')

        # Initialize components
        self.collaborative_model = None
        self.content_model = None
        self.user_interaction_matrix = None
        self.content_similarity_matrix = None
        self.scaler = MinMaxScaler()

        # Prepare the data
        self._prepare_data()

    def _prepare_data(self):
        """Enhanced data preparation with better weighting and normalization"""
        # Convert timestamps to datetime
        self.likes['liked_at'] = pd.to_datetime(self.likes['liked_at'])
        self.views['viewed_at'] = pd.to_datetime(self.views['viewed_at'])
        self.inspire['inspired_at'] = pd.to_datetime(self.inspire['inspired_at'])
        self.ratings['rated_at'] = pd.to_datetime(self.ratings['rated_at'])

        # Calculate recency weights
        now = pd.Timestamp.now()

        def calculate_recency_weight(timestamp):
            days_old = (now - timestamp).days
            return np.exp(-days_old / 365)  # Exponential decay over a year

        # Enhanced interaction weights with recency
        self.likes['interaction_weight'] = (
            self.likes['liked_at'].apply(calculate_recency_weight) * 1.0
        )
        self.views['interaction_weight'] = (
            self.views['viewed_at'].apply(calculate_recency_weight) * 0.5
        )
        self.inspire['interaction_weight'] = (
            self.inspire['inspired_at'].apply(calculate_recency_weight) * 0.8
        )

        # Normalize ratings with recency
        self.ratings['interaction_weight'] = (
            (self.ratings['rating_percent'] / 100.0) *
            self.ratings['rated_at'].apply(calculate_recency_weight)
        )

        # Combine all interactions
        interactions = pd.concat([
            self.likes[['user_id', 'post_id', 'interaction_weight']],
            self.views[['user_id', 'post_id', 'interaction_weight']],
            self.ratings[['user_id', 'post_id', 'interaction_weight']],
            self.inspire[['user_id', 'post_id', 'interaction_weight']]
        ])

        # Aggregate interactions with sophisticated weighting
        self.interactions = interactions.groupby(['user_id', 'post_id']).agg({
            'interaction_weight': lambda x: np.max(x) * (1 + np.log1p(len(x)))  # Boost multiple interactions
        }).reset_index()

        # Normalize interaction weights
        self.interactions['interaction_weight'] = self.scaler.fit_transform(
            self.interactions[['interaction_weight']]
        )

        # Create user-item interaction matrix
        self.user_interaction_matrix = self.interactions.pivot(
            index='user_id',
            columns='post_id',
            values='interaction_weight'
        ).fillna(0)

    def train_collaborative_filtering(self, n_components=100):
        """Enhanced collaborative filtering with more components and regularization"""
        # Add regularization to avoid overfitting
        svd = TruncatedSVD(n_components=n_components)
        self.collaborative_model = svd
        user_item_matrix = self.user_interaction_matrix.copy()

        # Add regularization term
        reg_factor = 0.1
        user_item_matrix += reg_factor * np.mean(user_item_matrix.values)

        self.user_features = svd.fit_transform(user_item_matrix)

    def _extract_text_features(self, text):
        """Enhanced text feature extraction"""
        if pd.isna(text) or text == '':
            return ''
        # Convert to lowercase and remove special characters
        text = str(text).lower()
        text = ''.join(c for c in text if c.isalnum() or c.isspace())
        return text

    def train_content_based(self):
        """Enhanced content-based filtering with better feature engineering"""
        # Extract features from video metadata with better error handling
        self.summary['category'] = self.summary['category'].apply(
            lambda x: literal_eval(x) if isinstance(x, str) else {'name': 'unknown', 'description': ''}
        )

        # Extract more features from category
        self.summary['category_name'] = self.summary['category'].apply(
            lambda x: self._extract_text_features(x.get('name', 'unknown'))
        )
        self.summary['category_desc'] = self.summary['category'].apply(
            lambda x: self._extract_text_features(x.get('description', ''))
        )

        # Clean and combine text features
        self.summary['title_clean'] = self.summary['title'].apply(self._extract_text_features)
        self.summary['slug_clean'] = self.summary['slug'].apply(
            lambda x: self._extract_text_features(str(x).replace('-', ' '))
        )

        # Combine all features with weights
        self.summary['content_features'] = (
            self.summary['title_clean'] + ' ' * 3 +  # Give more weight to title
            self.summary['category_name'] + ' ' * 2 +  # Give medium weight to category
            self.summary['category_desc'] + ' ' +
            self.summary['slug_clean']
        )

        # Create TF-IDF matrix with better parameters
        tfidf = TfidfVectorizer(
            stop_words='english',
            min_df=2,  # Minimum document frequency
            max_df=0.95,  # Maximum document frequency
            ngram_range=(1, 2)  # Include bigrams
        )
        tfidf_matrix = tfidf.fit_transform(self.summary['content_features'])

        # Calculate content similarity with normalization
        self.content_similarity_matrix = cosine_similarity(tfidf_matrix)
        self.content_model = tfidf

    def get_collaborative_recommendations(self, user_id, n_recommendations=10):
        """Get collaborative filtering recommendations with enhanced scoring"""
        if user_id not in self.user_interaction_matrix.index:
            return []

        user_idx = list(self.user_interaction_matrix.index).index(user_id)
        user_vector = self.user_features[user_idx:user_idx+1]
        predicted_ratings = user_vector @ self.collaborative_model.components_

        # Get user's interaction history
        already_interacted = set(self.interactions[self.interactions['user_id'] == user_id]['post_id'])
        post_ids = self.user_interaction_matrix.columns

        # Enhanced scoring with popularity adjustment
        post_popularity = self.user_interaction_matrix.sum()
        popularity_factor = 0.1

        recommendations = []
        for idx in (-predicted_ratings).argsort()[0]:
            post_id = post_ids[idx]
            if post_id not in already_interacted:
                # Adjust score with popularity
                score = predicted_ratings[0][idx]
                popularity_boost = popularity_factor * (post_popularity[post_id] / post_popularity.max())
                adjusted_score = score + popularity_boost

                recommendations.append((post_id, adjusted_score))
                if len(recommendations) >= n_recommendations:
                    break

        return recommendations

    def get_content_recommendations(self, post_id, n_recommendations=10):
        """Enhanced content-based recommendations"""
        if post_id not in self.summary['id'].values:
            return []

        idx = self.summary[self.summary['id'] == post_id].index[0]
        similar_scores = list(enumerate(self.content_similarity_matrix[idx]))

        # Add diversity penalty
        def diversity_penalty(scores, threshold=0.95):
            penalized_scores = []
            selected = set()

            for i, score in sorted(scores, key=lambda x: x[1], reverse=True):
                if len(penalized_scores) >= n_recommendations:
                    break

                # Check similarity with already selected items
                too_similar = False
                for j in selected:
                    if self.content_similarity_matrix[i, j] > threshold:
                        too_similar = True
                        break

                if not too_similar:
                    penalized_scores.append((i, score))
                    selected.add(i)

            return penalized_scores

        diverse_scores = diversity_penalty(similar_scores)

        recommendations = []
        for i, score in diverse_scores:
            if i != idx:  # Exclude the input post
                recommendations.append((self.summary.iloc[i]['id'], score))
                if len(recommendations) >= n_recommendations:
                    break

        return recommendations

    def get_hybrid_recommendations(self, user_id, n_recommendations=10, collaborative_weight=0.7):

        # Get base recommendations
        cf_recs = self.get_collaborative_recommendations(user_id, n_recommendations*2)

        # Get user's interaction history
        user_interactions = self.interactions[self.interactions['user_id'] == user_id]
        interaction_count = len(user_interactions)

        # Adjust weights based on user interaction history
        if interaction_count < 5:
            collaborative_weight = 0.3  # Rely more on content-based for new users
        elif interaction_count < 20:
            collaborative_weight = 0.5  # Balanced for moderately active users

        # Get content-based recommendations from recent interactions
        recent_interactions = user_interactions.sort_values('interaction_weight', ascending=False)

        content_scores = {}
        for _, interaction in recent_interactions.head(5).iterrows():
            cb_recs = self.get_content_recommendations(interaction['post_id'], n_recommendations*2)
            for post_id, score in cb_recs:
                if post_id in content_scores:
                    content_scores[post_id] = max(content_scores[post_id], score)
                else:
                    content_scores[post_id] = score

        # Combine scores with sophisticated blending
        hybrid_scores = {}
        for post_id, cf_score in cf_recs:
            # Check if post_id exists in summary before including it
            if post_id in self.summary['id'].values:
                cb_score = content_scores.get(post_id, 0)
                # Geometric mean for better balance
                hybrid_scores[post_id] = (
                    (collaborative_weight * cf_score ** 2 +
                    (1 - collaborative_weight) * cb_score ** 2) ** 0.5
                )

        # Add remaining content-based recommendations
        for post_id, cb_score in content_scores.items():
            # Check if post_id exists in summary before including it
            if post_id not in hybrid_scores and post_id in self.summary['id'].values:
                cf_score = 0
                hybrid_scores[post_id] = (
                    (collaborative_weight * cf_score ** 2 +
                    (1 - collaborative_weight) * cb_score ** 2) ** 0.5
                )

        # Sort and return top recommendations with diversity check
        recommendations = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
        diverse_recs = []
        seen_categories = set()

        for post_id, score in recommendations:
            # Safe category lookup with error handling
            post_data = self.summary[self.summary['id'] == post_id]
            if len(post_data) > 0:  # Check if post exists in summary
                post_category = post_data['category_name'].iloc[0]
                if post_category not in seen_categories or len(seen_categories) >= 3:
                    diverse_recs.append((post_id, score))
                    seen_categories.add(post_category)
                    if len(diverse_recs) >= n_recommendations:
                        break

        return diverse_recs

    def get_cold_start_recommendations(self, category_id=None, mood=None, n_recommendations=10):
        """Enhanced cold start recommendations"""
        filtered_posts = self.summary.copy()

        if category_id:
            filtered_posts = filtered_posts[filtered_posts['category'].apply(
                lambda x: x.get('id') == category_id if isinstance(x, dict) else False
            )]

        # Enhanced scoring with multiple factors
        filtered_posts['popularity_score'] = (
            0.3 * (filtered_posts['upvote_count'] / filtered_posts['upvote_count'].max()) +
            0.3 * (filtered_posts['view_count'] / filtered_posts['view_count'].max()) +
            0.2 * (filtered_posts['rating_count'] / filtered_posts['rating_count'].max()) +
            0.2 * (filtered_posts['comment_count'] / (filtered_posts['comment_count'].max() + 1))
        )

        # Add diversity to recommendations
        diverse_recommendations = []
        seen_categories = set()

        for _, post in filtered_posts.sort_values('popularity_score', ascending=False).iterrows():
            post_category = post['category_name']
            if post_category not in seen_categories or len(seen_categories) >= 3:
                diverse_recommendations.append(post['id'])
                seen_categories.add(post_category)
                if len(diverse_recommendations) >= n_recommendations:
                    break

        return diverse_recommendations


    def evaluate(self):
        print("Evaluating Recommendation System Performance...")
        interactions = self.interactions.copy()
        train_data, test_data = train_test_split(
            interactions,
            test_size=0.2,
            random_state=42,
            stratify=interactions['user_id'].map(interactions['user_id'].value_counts())
        )

        # Create training interaction matrix
        train_matrix = train_data.pivot(
            index='user_id',
            columns='post_id',
            values='interaction_weight'
        ).fillna(0)

        # Train SVD model and make predictions
        svd = TruncatedSVD(n_components=100)
        user_features = svd.fit_transform(train_matrix)
        predicted_matrix = user_features @ svd.components_

        # Collect true and predicted values
        y_true = []
        y_pred = []

        for _, row in test_data.iterrows():
            if row['user_id'] in train_matrix.index and row['post_id'] in train_matrix.columns:
                user_idx = list(train_matrix.index).index(row['user_id'])
                post_idx = list(train_matrix.columns).index(row['post_id'])
                actual = row['interaction_weight']
                predicted = predicted_matrix[user_idx, post_idx]

                y_true.append(actual)
                y_pred.append(predicted)

        # Calculate metrics
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        # Print results
        print("Evaluation Metrics:")
        print(f"  Mean Absolute Error (MAE): {mae:.4f}")
        print(f"  Root Mean Square Error (RMSE): {rmse:.4f}")

        return {
            'mae': mae,
            'rmse': rmse
        }

def main():
    # Initialize recommender
    print("Initializing Enhanced Video Recommender...")
    recommender = EnhancedVideoRecommender()

    # Train models
    print("\nTraining recommendation models...")
    recommender.train_collaborative_filtering()
    recommender.train_content_based()

    # Example recommendations for a user
    user_id = 1  # example user

    print("\nGenerating recommendations for example user (ID: 1)...")
    print("\nCollaborative Filtering Recommendations:")
    cf_recs = recommender.get_collaborative_recommendations(user_id)
    print(cf_recs[:10])

    print("\nContent-Based Recommendations for a post (ID: 26):")
    post_id = 26  # example post
    cb_recs = recommender.get_content_recommendations(post_id)
    print(cb_recs[:10])

    print("\nHybrid Recommendations:")
    hybrid_recs = recommender.get_hybrid_recommendations(user_id)
    print(hybrid_recs[:10])

    print("\nCold Start Recommendations (Category ID: 4):")
    cold_start_recs = recommender.get_cold_start_recommendations(category_id=4)
    print(cold_start_recs[:10])

    # Comprehensive evaluation
    print("\nPerforming comprehensive system evaluation...")
    metrics = recommender.evaluate()

if __name__ == "__main__":
    main()

Initializing Enhanced Video Recommender...

Training recommendation models...

Generating recommendations for example user (ID: 1)...

Collaborative Filtering Recommendations:
[(390, 0.003864398911986448), (659, 0.0029823954072346823), (648, 0.0018255343763099055), (452, 0.004429089923121682), (714, 0.0035190370399566166), (847, 0.015262431567528715), (654, 0.002645882325836071), (867, 0.0016950859119149269), (1007, 0.0030451027479774405), (656, 0.0016486570159511315)]

Content-Based Recommendations for a post (ID: 26):
[(1150, 0.5262858096812432), (1326, 0.4327579860941045), (12, 0.27017009338311854), (785, 0.18392728017752388), (1000, 0.1804173105441492), (983, 0.1662299718485631), (672, 0.16533419496103924), (1176, 0.15803451695070095), (657, 0.15413112272475005)]

Hybrid Recommendations:
[(12, 0.5477225575051663), (88, 0.06170342253283474), (1071, 0.003207120933859767), (714, 0.0029442376232244973), (1007, 0.0025477157459217917), (659, 0.0024952510205520743), (156, 0.00232738219443

In [5]:
from flask import Flask, request, jsonify
from werkzeug.serving import run_simple
import threading
import time

app = Flask(__name__)
recommender = EnhancedVideoRecommender()

# Train models on startup
print("Training recommendation models...")
recommender.train_collaborative_filtering()
recommender.train_content_based()

@app.route('/feed', methods=['GET'])
def get_recommendations():
    try:
        # Required parameter
        username = request.args.get('username')
        if not username:
            return jsonify({
                'success': False,
                'error': 'Username is required'
            }), 400

        # Optional parameters
        category_id = request.args.get('category_id')
        mood = request.args.get('mood')

        # Get user information
        user = recommender.users[recommender.users['username'] == username]

        # Generate recommendations based on available parameters
        if user.empty:
            # Cold start case - new user
            recommendations = recommender.get_cold_start_recommendations(
                category_id=int(category_id) if category_id else None,
                mood=mood,
                n_recommendations=10
            )
            user_status = 'new'
        else:
            # Existing user - use hybrid recommendations
            user_id = user.iloc[0]['id']
            recommendations = recommender.get_hybrid_recommendations(
                user_id=user_id,
                n_recommendations=10
            )
            recommendations = [rec[0] for rec in recommendations]
            user_status = 'existing'

        # Get full post details
        recommended_posts = recommender.summary[recommender.summary['id'].isin(recommendations)]

        # Filter by category if provided
        if category_id and not user.empty:
            category_id = int(category_id)
            recommended_posts = recommended_posts[
                recommended_posts['category'].apply(
                    lambda x: x.get('id') == category_id if isinstance(x, dict) else False
                )
            ]

        # Ensure exactly 10 recommendations
        recommended_posts = recommended_posts.head(10)

        return jsonify({
            'success': True,
            'user_status': user_status,
            'count': len(recommended_posts),
            'parameters_used': {
                'username': username,
                'category_id': category_id,
                'mood': mood
            },
            'recommendations': recommended_posts.to_dict('records')
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e),
            'message': 'Error generating recommendations'
        }), 500

def run_flask():
    app.run(host='localhost', port=5000, debug=True, use_reloader=False)

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait for server to start
time.sleep(2)
print("API server is running at http://localhost:5000")
print("\nAvailable endpoint variations:")
print("1. /feed?username=your_username&category_id=category_id&mood=user_mood")
print("2. /feed?username=your_username&category_id=category_id")
print("3. /feed?username=your_username")

Training recommendation models...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit


API server is running at http://localhost:5000

Available endpoint variations:
1. /feed?username=your_username&category_id=category_id&mood=user_mood
2. /feed?username=your_username&category_id=category_id
3. /feed?username=your_username


In [6]:
import requests

# Test all three endpoint variations
base_url = "http://localhost:5000/feed"

# 1. Full parameters
response = requests.get(f"{base_url}?username=afrobeezy&category_id=2&mood=motivated")
print("Full parameters response:")
print(response.json())

# 2. Username and category
response = requests.get(f"{base_url}?username=afrobeezy&category_id=2")
print("\nUsername and category response:")
print(response.json())

# 3. Username only
response = requests.get(f"{base_url}?username=afrobeezy")
print("\nUsername only response:")
print(response.json())

INFO:werkzeug:127.0.0.1 - - [13/Dec/2024 07:18:20] "GET /feed?username=afrobeezy&category_id=2&mood=motivated HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Dec/2024 07:18:20] "GET /feed?username=afrobeezy&category_id=2 HTTP/1.1" 200 -


Full parameters response:
{'count': 6, 'parameters_used': {'category_id': 2, 'mood': 'motivated', 'username': 'afrobeezy'}, 'recommendations': [{'average_rating': 0, 'baseToken': "{'address': '', 'name': '', 'symbol': '', 'image_url': ''}", 'bookmarked': False, 'category': {'count': 540, 'description': 'All the best vibes!', 'id': 2, 'image_url': 'https://assets.socialverseapp.com/categories/a6861bd94e900fe7b6a88b72fd1975ea', 'name': 'Vible'}, 'category_desc': 'all the best vibes', 'category_name': 'vible', 'chain_id': nan, 'chart_url': nan, 'comment_count': 0, 'content_features': 'why fit in   vible  all the best vibes why fit in', 'contract_address': nan, 'created_at': 1698096725000, 'exit_count': 0, 'first_name': 'Michael', 'following': True, 'gif_thumbnail_url': nan, 'id': 12, 'identifier': 'iGqBOIW', 'is_locked': False, 'last_name': 'Dadzie', 'picture_url': 'https://assets.socialverseapp.com/profile/afrobeezy1704186478image_cropper_B57BCBB5-40B6-4B0A-8710-940B7292DC11-2518-0000020

INFO:werkzeug:127.0.0.1 - - [13/Dec/2024 07:18:20] "GET /feed?username=afrobeezy HTTP/1.1" 200 -



Username only response:
{'count': 10, 'parameters_used': {'category_id': None, 'mood': None, 'username': 'afrobeezy'}, 'recommendations': [{'average_rating': 0, 'baseToken': "{'address': '', 'name': '', 'symbol': '', 'image_url': ''}", 'bookmarked': False, 'category': {'count': 540, 'description': 'All the best vibes!', 'id': 2, 'image_url': 'https://assets.socialverseapp.com/categories/a6861bd94e900fe7b6a88b72fd1975ea', 'name': 'Vible'}, 'category_desc': 'all the best vibes', 'category_name': 'vible', 'chain_id': nan, 'chart_url': nan, 'comment_count': 0, 'content_features': 'why fit in   vible  all the best vibes why fit in', 'contract_address': nan, 'created_at': 1698096725000, 'exit_count': 0, 'first_name': 'Michael', 'following': True, 'gif_thumbnail_url': nan, 'id': 12, 'identifier': 'iGqBOIW', 'is_locked': False, 'last_name': 'Dadzie', 'picture_url': 'https://assets.socialverseapp.com/profile/afrobeezy1704186478image_cropper_B57BCBB5-40B6-4B0A-8710-940B7292DC11-2518-00000201B79